Using 100 science fiction novels from project gutenberg. 

Source: https://www.gutenberg.org/wiki/Science_Fiction_(Bookshelf)

In [741]:
import nltk
import pandas as pd
import numpy as np
import books_files as books 
from urllib import request

In [742]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def text_cleaner(text_data):
    text = re.sub("[^a-zA-Z]",' ',text_data)
    text = text.lower()
    text = text.split()
    lmz = WordNetLemmatizer()
    text = [lmz.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

In [743]:
from io import StringIO

book_text = []
authors = []

for k,v in books.url_dict.items():
    for url in v:
        response = request.urlopen(url)
        raw = response.read().decode('utf8')
        s = StringIO(raw)
        text_lines = []
        for i,line in enumerate(s):
            text_lines.append(line)
            if "START OF THIS PROJECT" in line:
                start_index = (i+1)
            if "END OF THIS PROJECT" in line:
                end_index = i
        book_text.append(' '.join(text_lines[start_index:end_index]))
        authors.append(k)

In [744]:
# 100 different books
print(len(book_text))

100


In [745]:
# 16 different authors
print(len(set(authors)))

16


In [746]:
text_df = pd.DataFrame()
text_df["Text"] = book_text
text_df["Author"] = authors
text_df["Author_ID"] = text_df["Author"].factorize()[0]
text_df.head(3)

,Text,Author,Author_ID
0,"\r\n \r\n \r\n \r\n Produced by Greg Weeks, Ma...",Anderson,0
1,\r\n \r\n \r\n \r\n Produced by Sankar Viswana...,Anderson,0
2,\r\n \r\n \r\n \r\n Produced by Sankar Viswana...,Anderson,0


In [ ]:
# bag of words
w2v = []
corpus = []
for text in book_text:
    clean_text = text_cleaner(text)
    corpus.append(clean_text)
    clean_text = nltk.word_tokenize(clean_text)
    w2v.append(clean_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_v = TfidfVectorizer(max_df=0.5,
                             min_df =2,
                             stop_words="english",
                             lowercase=False,
                             use_idf=True,
                             norm=u'l2',
                             smooth_idf=True
                            )

In [ ]:
X = tfidf_v.fit_transform(corpus)
y = text_df.iloc[:,2].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
words = tfidf_v.get_feature_names()

Going to use cosine similarity to generate a measure of similarity between each book in the corpus (tf-idf matrix). We are subtracting the cosine similarity from 1 to get the cosine distance which will be used later on to plot the clusters on a euclidean 2D plane. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(X_train)

## KMeans Cluster

### Finding optimal number of clusters

In [ ]:
# elbow method
from sklearn.cluster import KMeans
wcss = []
n_clusters = range(1,10)
for i in n_clusters:
    kmeans = KMeans(n_clusters=i,random_state=42)
    kmeans.fit(X_train)
    wcss.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (23,13)
plt.plot(n_clusters,wcss)
plt.title("The Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.xticks(n_clusters)
plt.show()

In [ ]:
# 6 clusters seems optimal
num_clusters = 6
km = KMeans(n_clusters=num_clusters,random_state=42)

In [ ]:
km.fit(X_train)
clusters = km.labels_.tolist()

In [ ]:
books = {"author":y_train,"book_text":X_train,"cluster":clusters}
frame = pd.DataFrame(books,index=[clusters],columns=["author","book_text","cluster"])

In [ ]:
author_id = {}
for author in list(text_df["Author_ID"]):
    if author not in author_id:
        author_id[author] = text_df.loc[text_df["Author_ID"]==author,"Author"].iloc[0]
author_id

In [ ]:
frame["cluster"].value_counts()

## Getting top 10 words and authors per cluster

In [ ]:
from __future__ import print_function

print("Authors per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
cluster_names = {}

for i in range(num_clusters):
    cluster_names[i] = ""
    print("Cluster %d:"%i)
    print("Top 10 words:")
    for ind in order_centroids[i,:10]:
        cluster_names[i] += ("{} ".format(str(words[ind])))
        print("%s"%words[ind])
    cluster_names[i] = ','.join(cluster_names[i].split())
    print()
    print("Cluster %d authors:" % i, end='')
    for author in frame.loc[i]['author'].values.tolist():
        print(' %s,' % author_id[author], end='')
    print() #add whitespace
    print() #add whitespace

## K-Means Clusters visual (dimension reduction)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

X_norm = normalize(dist)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_norm)

In [ ]:
variance_explained = pca.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components: ",total_variance*100)

In [ ]:
xs,ys = X_pca[:,0],X_pca[:,1]

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#ffe059', 1: '#9cff07', 2: '#ff0707', 3: '#07ffc1', 4: '#a407ff',5: '#ff07bc'}

#cluster_names = {0: 'joe, kenzie, guy, instructor, chessman, chief, watson, brett, buckner, baron', 
                 #1: 'johnny, mead, kennedy, captain, geoffrey, dane, commodore, ron, aunt, wilkins', 
                 #2: 'nipe, stanton, mannheim, bart, yoritomo, martin, colonel, farnsworth, rat, wang', 
                 #3: 'colonel, guesser, zen, porter, senator, gibson, bending, charlie, parker, stoker', 
                 #4: 'gordon, thompson, ross, muller, mar, doctor, henry, fisher, doc, reactor',
                 #5: 'martian, robot, robert, kitty, presently, miller, animal, redwood, beast, blackie'}

In [ ]:
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, author=[author_id[i] for i in y_train])) 

groups = df.groupby('label')

In [ ]:
fig, ax = plt.subplots(figsize=(21, 10))

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='False',      # ticks along the bottom edge are off
        top='False',         # ticks along the top edge are off
        labelbottom='False')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='False',      # ticks along the bottom edge are off
        top='False',         # ticks along the top edge are off
        labelleft='False')

ax.legend(numpoints=1)  #show legend with only 1 point

for i in range(len(df)):
    ax.text(df.loc[i]['x'], df.loc[i]['y'], df.loc[i]['author'], size=8)  

plt.show()

## Hierarchical Cluster visual

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist)

fig, ax = plt.subplots(figsize=(15, 25)) # set size

ax = dendrogram(linkage_matrix, orientation="right", labels=[author_id[i] for i in y_train]);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='False',      # ticks along the bottom edge are off
    top='False',         # ticks along the top edge are off
    labelbottom='False')
plt.tick_params(labelsize=20)
plt.tight_layout() #show plot with tight layout

## Classifying book texts by author (Supervised Learning Application)

We will attempt to use supervised and unsupervised learning techniques to identify the authors by their respective book texts. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# use PCA to generate features (features should be less than 100, only have 100 different books)
#X_norm_features = normalize(X.toarray())
pca = PCA(0.95)
X_features = pca.fit_transform(X.toarray())

In [ ]:
ypred_km = km.predict(X)

In [ ]:
features_rough = np.concatenate((X_features,ypred_km[:,None]),axis=1)

In [ ]:
features_df = pd.DataFrame(features_rough)

In [ ]:
features_df.head(3)

In [ ]:
dummy = pd.get_dummies(features_df[90])
dummy.head(3)

In [ ]:
features_df = pd.concat([features_df,dummy],axis=1)

In [ ]:
features_df = features_df.drop([90.0,5.0],axis=1)

In [ ]:
features_df.head(3)

In [ ]:
X = features_df.iloc[:,:-1].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
models=[
    LogisticRegression(random_state=42),
    RandomForestClassifier(n_estimators=300,max_depth=3,random_state=42),
    SVC(),
    #MultinomialNB(),
    XGBClassifier()
]

In [ ]:
CV = 6
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model,X,y,scoring="accuracy",cv=CV)
    
    for fold_idx,accuracy in enumerate(accuracies):
        entries.append((model_name,fold_idx,accuracy))

In [ ]:
cv_df = pd.DataFrame(entries,columns=["model_name","fold_idx","accuracy"])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

### Evaluating Classification Model Performance (XGBClassifier)

In [ ]:
xgc = XGBClassifier()

xgc.fit(X_train,y_train)
ypred_xgc = xgc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(y_test,ypred_xgc)

fig,ax = plt.subplots(figsize=(10,10))

sns.heatmap(conf_mat,annot=True,fmt='d',xticklabels=[v for k,v in author_id.items()],yticklabels=[v for k,v in author_id.items()])
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()

In [ ]:
# classification report for each class (XGC)
from sklearn import metrics

print(metrics.classification_report(y_test,ypred_xgc,target_names=[v for k,v in author_id.items()]))

Next steps:

1. Get a benchmark of all supervised learning models (fitting and comparing accuracy scores)

2. Evaluate the best model (most accurate, best score) using confusion matrix

3. Explore sklearn's classification_report module

4. Use word2vec for feature engineering, visualize & interpret clusters made by word2vec (TSNE & Affinity Propagation)

5. Extra: Apply supervised learning models to word2vec features